In [ ]:
!pip install onnxruntime
!pip install tf2onnx

In [7]:
import time
import torch
import numpy as np
import onnxruntime as ort
import tensorflow as tf
from tensorflow.keras.applications import MobileNetV2
import pandas as pd

In [5]:
# Load a pre-trained MobileNetV2 model from TensorFlow
model = MobileNetV2(weights="imagenet")
input_shape = model.input_shape[1:]

def benchmark_tf(model, input_data, runs=100):
    """Benchmark TensorFlow inference."""
    infer = model
    times = []
    outputs = []
    for _ in range(runs):
        start = time.time()
        output = infer.predict(input_data)
        end = time.time()
        times.append(end - start)
        outputs.append(output)
    return times, outputs[-1]

# Convert the model to ONNX format
import tf2onnx
onnx_model_path = "mobilenetv2.onnx"
spec = (tf.TensorSpec((None, *input_shape), tf.float32, name="input"),)
model_proto, _ = tf2onnx.convert.from_keras(model, input_signature=spec, opset=13)
with open(onnx_model_path, "wb") as f:
    f.write(model_proto.SerializeToString())

def benchmark_onnx(model_path, input_data, runs=100):
    """Benchmark ONNX inference."""
    session = ort.InferenceSession(model_path, providers=["CPUExecutionProvider"])
    input_name = session.get_inputs()[0].name
    times = []
    outputs = []
    for _ in range(runs):
        start = time.time()
        output = session.run(None, {input_name: input_data})[0]
        end = time.time()
        times.append(end - start)
        outputs.append(output)
    return times, outputs[-1]

# Generate random input data
input_data = np.random.rand(1, *input_shape).astype(np.float32)

# Run multiple benchmarks
num_experiments = 10
tf_results = []
onxx_results = []
tf_output = None
onnx_output = None

for _ in range(num_experiments):
    tf_times, tf_output = benchmark_tf(model, input_data)
    onnx_times, onnx_output = benchmark_onnx(onnx_model_path, input_data)
    tf_results.extend(tf_times)
    onxx_results.extend(onnx_times)

# Compare outputs
difference = np.max(np.abs(tf_output - onnx_output))
print(f"Max difference between TensorFlow and ONNX outputs: {difference}")

# Export results
df = pd.DataFrame({
    "TensorFlow": tf_results,
    "ONNX": onxx_results
})
df.to_csv("benchmark_results.csv", index=False)

print("Benchmark results saved to benchmark_results.csv")


14536120/14536120 ━━━━━━━━━━━━━━━━━━━━ 0s 0us/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 2s 2s/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 111ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 113ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 137ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 112ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 117ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 115ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 77ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 74ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 89ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 97ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 92ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 85ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 104ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 75ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 73ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 73ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 71ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 75ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 75ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 73ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 72ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 75ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 74ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 75ms

In [18]:
# Convert TensorFlow output to Torch tensor for comparison
tf_output_torch = torch.tensor(tf_output)
onnx_output_torch = torch.tensor(onnx_output)

# Check if outputs match
try:
    torch.testing.assert_close(tf_output_torch, onnx_output_torch)
    print("TensorFlow and ONNX Runtime output matched!")
except AssertionError:
    print("TensorFlow and ONNX Runtime output do not match!")

TensorFlow and ONNX Runtime output matched!


In [17]:
# Load the benchmark results
df = pd.read_csv("benchmark_results.csv")

# Compute mean and standard deviation
summary = df.agg(['mean', 'std'])

# Format results in "mean ± std" format
formatted_results = summary.apply(lambda x: f"{x['mean']:.4f} ± {x['std']:.4f}", axis=0)

# Save to a new CSV
formatted_results.to_csv("formatted_benchmark_results.csv", header=False)

df_formatted = pd.read_csv("formatted_benchmark_results.csv")
print(df_formatted)

  TensorFlow  0.1977 ± 0.0839
0       ONNX  0.0151 ± 0.0045
